In [ ]:
import pandas as pd
import pygal

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from IPython.display import HTML
from IPython.display import SVG

In [ ]:
undefined_color = []
for i in range(100):
    undefined_color.append("#ffffff")
undefined_color = tuple(undefined_color)

undefined_color_style = pygal.style.Style(colors=undefined_color)

In [ ]:
def grande_region_to_vraie_region(code):
    if code == 5:
        return ('01','02','03','04','05','06')
    elif code == 11:
        return ('11',)
    elif code == 24:
        return ('24',)
    elif code == 27:
        return ('26', '43')
    elif code == 28:
        return ('23', '25')
    elif code == 32:
        return ('22', '31')
    elif code == 44:
        return ('21', '41', '42',)
    elif code == 52:
        return ('52',)
    elif code == 53:
        return ('53',)
    elif code == 75:
        return ('54', '72', '74')
    elif code == 76:
        return ('73', '91')
    elif code == 84:
        return ('82', '83')
    elif code == 93:
        return ('93', '94')
    elif code == 99 or code == 0:
        return ()
    else:
        raise NotImplementedError

In [ ]:
# population 2019
# population_2019 = ((84,8042936),(27,2805580),(53,3354854),(24,2573180),(94,340440),(44,5556219),(32,6004947),(11,12262544),(28,3325032),(75,6010289),(76,5933185),(52,3806461),(93,5081101),(5,384239+364508+281678+861210+269579))
population_2019 = ((84,8042936),(27,2805580),(53,3354854),(24,2573180),(44,5556219),(32,6004947),(11,12262544),(28,3325032),(75,6010289),(76,5933185),(52,3806461),(93,5081101+340440),(5,384239+364508+281678+861210+269579))
population_2019 = dict((x, y) for x, y in population_2019)

In [ ]:
df_list = []
for y in range(2017, 2021+1):
    df = pd.read_csv(f'raw/OPEN_MEDIC_{y}.zip', sep=';', compression='zip', encoding='ansi', usecols=['BEN_REG', 'REM'])
    df['REM'] = df['REM'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)
    df_list.append(dict(df.groupby('BEN_REG')['REM'].sum()))
del df

In [ ]:
cmap_eur = matplotlib.cm.get_cmap('YlOrRd')

gradient = np.linspace(0, 1, 100)
gradient = np.vstack([gradient] * 4)

plt.figure(figsize=(12,1))
plt.title("€ [225, 350]")
fig = plt.imshow(gradient, cmap=cmap_eur)
fig.axes.get_yaxis().set_visible(False)
plt.savefig("legende.png")

In [ ]:
for y in range(2017, 2021+1):
    i = y-2017
    
    # anonymisé / non ciblé
    total = sum(df_list[i].values())
    somme = 0
    for k in df_list[i].keys():
        regions = grande_region_to_vraie_region(k)
        if regions == ():
            somme += df_list[i][k]
    undefined = '{:.2f}%'.format(round((somme / total)*100, 2))
    somme = "{0:,} €".format(int(somme)).replace(',',' ')
    total = "{0:,} €".format(int(total)).replace(',',' ')
    
    # carte
    chart = pygal.maps.fr.Regions(show_legend=False, style=undefined_color_style)
    chart.title = f"{y} - dépense de l'assurance maladie par personne (médicaments)\n non ciblé : {somme} ({undefined})\n total : {total}"
    
    for k in population_2019.keys():
        v = df_list[i][k] / population_2019[k]

        regions = grande_region_to_vraie_region(k)
        
        r, g, b, a = cmap_eur((((v-225)*2)/250))
        r *= 255
        g *= 255
        b *= 255
        
        for reg in regions:
            chart.add('{:.2f}€'.format(round(v, 2)), [{'value':f"{reg}", 'color': f'rgb({r},{g},{b})'}])
    svg = chart.render()
    display(SVG(svg))
    chart.render_to_file(f'{y}.svg')